# Dashboard Building for Car Sales Data: Insights from mobil123.com

## Importing Libraries

In [2]:
!pip install requests
import requests

!pip install beautifulsoup4
from bs4 import BeautifulSoup

import pandas as pd
import re

pd.set_option('display.max_rows', 2000)

## Data Scraping

In [190]:
mobil = []

#for some reason the web restricts me from scraping it, it might think i'm a bot, so i'm using user-agent header to verify i'm a real user.
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36" }
#got this from here: https://www.useragentstring.com/index.php


for i in range(1,1054):#pages total 1053 pages
  url = f"https://www.mobil123.com/mobil-dijual/indonesia?max_price=200000000&page_size=25&page_number={i}" #data is taken on 22/07/2024 with the filter price 0 - 200M Rupiah
  response = requests.get(url,headers=headers)
  soup = BeautifulSoup(response.content, 'html.parser')
  main = soup.find('section', id="classified-listings-result")
  articles = main.find_all('article')

  for article in articles:
    #title for each car sale
    title = article.find('h2', class_='listing__title')
    a_tag = title.find('a')
    title = a_tag.text.strip() if title else '-' #getting only the text and remove whitespace, also NA handling

    #link
    link = a_tag.get('href') if a_tag else '-'
    
    #car price
    price = article.find('div', class_='two-thirds')
    price = price.text.strip() if price else '-'

    #car name
    car = article.find('div',class_="listing__rating-model")
    car = car.text.strip() if car else '-'

    #car year
    year = article.get('data-year', '-')  # Default to '-' if attribute not found
    
    #car condition
    condition = article.get('data-ad-type', '-')  # Default to '-' if attribute not found

    divs = article.find_all('div',class_="item")
    items = [div.text.strip() for div in divs]

    #total km
    km = items[0] if items[0] else '-'
      
    #car transmission
    ts = items[1] if items[1] else '-'

    #location
    location = items[2] if items[2] else '-'


    
    mobil.append([title,car,condition,year,price,ts,km,location,link])
      


df = pd.DataFrame(mobil, columns=['Title', 'Car Name','Condition','Year','Price', 'Transmission', 'Total Kilometers','Location','Link'])
#df.to_excel('mobil.xlsx',index=False)
df

,Title,Car Name,Condition,Year,Price,Transmission,Total Kilometers,Location,Link
0,2018 Toyota Calya 1.2 G MPV,Toyota Calya,Used,2018,Rp 109.000.000,Automatic,90 - 95K KM,Banten,https://www.mobil123.com/dijual/toyota-calya-g...
1,2024 Suzuki XL7 1.5 ALPHA Hybrid Wagon - PROMO...,Suzuki XL7,New,2024,Rp 195.000.000,Automatic,0 - 5K KM,Jawa Barat,https://www.mobil123.com/dijual/suzuki-xl7-alp...
2,"2021 Suzuki XL7 1.5 BETA Wagon - 1,5",Suzuki XL7,Used,2021,Rp 195.000.000,Automatic,37538 KM,Banten,https://www.mobil123.com/dijual/suzuki-xl7-bet...
3,2011 Honda Jazz 1.5 Hatchback,Honda Jazz,Used,2011,Rp 115.000.000,Manual,130 - 135K KM,Banten,https://www.mobil123.com/dijual/honda-jazz-ban...
4,2019 Toyota Avanza 1.3 G MPV,Toyota Avanza,Used,2019,Rp 164.000.000,Manual,55 - 60K KM,Jawa Barat,https://www.mobil123.com/dijual/toyota-avanza-...
...,...,...,...,...,...,...,...,...,...
26320,2015 Ford EcoSport 1.5 Titanium SUV - Tdp.15jt...,Ford EcoSport,Used,2015,Rp 130.000.000,Automatic,65000 KM,DKI Jakarta,https://www.mobil123.com/dijual/ford-ecosport-...
26321,2020 Toyota Avanza 1.3 E MPV - Manual,Toyota Avanza,Used,2020,Rp 175.000.000,Manual,30 - 35K KM,Sumatera Utara,https://www.mobil123.com/dijual/toyota-avanza-...
26322,"2018 Mitsubishi Xpander 1.5 ULTIMATE Wagon - 1,5",Mitsubishi Xpander,Used,2018,Rp 192.000.000,Automatic,56205 KM,DKI Jakarta,https://www.mobil123.com/dijual/mitsubishi-xpa...
26323,"2019 Daihatsu Sigra 1.2 R Deluxe MPV - 1,2",Daihatsu Sigra,Used,2019,Rp 125.000.000,Automatic,79645 KM,Jawa Barat,https://www.mobil123.com/dijual/daihatsu-sigra...


### Saving the raw dataset

In [191]:
df.to_excel('mobil.xlsx',index=False)

In [3]:
df = pd.read_excel('mobil.xlsx')
df.head()

,Title,Car Name,Condition,Year,Price,Transmission,Total Kilometers,Location,Link
0,2018 Toyota Calya 1.2 G MPV,Toyota Calya,Used,2018,Rp 109.000.000,Automatic,90 - 95K KM,Banten,https://www.mobil123.com/dijual/toyota-calya-g...
1,2024 Suzuki XL7 1.5 ALPHA Hybrid Wagon - PROMO...,Suzuki XL7,New,2024,Rp 195.000.000,Automatic,0 - 5K KM,Jawa Barat,https://www.mobil123.com/dijual/suzuki-xl7-alp...
2,"2021 Suzuki XL7 1.5 BETA Wagon - 1,5",Suzuki XL7,Used,2021,Rp 195.000.000,Automatic,37538 KM,Banten,https://www.mobil123.com/dijual/suzuki-xl7-bet...
3,2011 Honda Jazz 1.5 Hatchback,Honda Jazz,Used,2011,Rp 115.000.000,Manual,130 - 135K KM,Banten,https://www.mobil123.com/dijual/honda-jazz-ban...
4,2019 Toyota Avanza 1.3 G MPV,Toyota Avanza,Used,2019,Rp 164.000.000,Manual,55 - 60K KM,Jawa Barat,https://www.mobil123.com/dijual/toyota-avanza-...


## Preprocessing

In [19]:
df2 = df.copy()

### Checking the values of each column

In [20]:
for column in df2.columns:
    print(df2[column].value_counts())
    print("")

Title
2024 Suzuki S-Presso 1.0 Hatchback                                                                                               735
2024 Toyota Agya 1.2 G Hatchback                                                                                                 386
2024 Toyota Calya 1.2 G MPV                                                                                                      267
2024 Honda Brio 1.2 E Satya Hatchback                                                                                            236
2024 Suzuki Ertiga 1.5 GX Hybrid MPV                                                                                             224
                                                                                                                                ... 
2023 Daihatsu Sigra 1.2 X MPV - DP MULAI 5 JUTA CICILAN 3 JTAAN                                                                    1
2023 Honda Brio 1.2 S Satya Hatchback                          

### Check for duplicated rows

In [21]:
df2.duplicated().sum()

53

In [22]:
#drop the duplicates and retain only the first occurence
df2 = df2.drop_duplicates(keep='first')
df2.reset_index(drop=True, inplace=True)
df2.duplicated().sum()

0

### Check for missing values

In [23]:
#the missing values were previously changed to "-"
df_na = (df2 == "-")
df_na.sum()

Title                  0
Car Name            3107
Condition              0
Year                   0
Price                  0
Transmission           0
Total Kilometers       0
Location               0
Link                   0
dtype: int64

In [24]:
#rows with missing values
df_temp = df2[df_na.any(axis=1)]
df_temp

,Title,Car Name,Condition,Year,Price,Transmission,Total Kilometers,Location,Link
25,2024 Suzuki S-Presso 1.0 Hatchback,-,New,2024,Rp 163.100.000,Automatic,0 - 5K KM,DKI Jakarta,https://www.mobil123.com/dijual/suzuki-s-press...
28,2023 Suzuki S-Presso 1.0 Hatchback,-,New,2023,Rp 152.300.000,Manual,0 - 5K KM,DKI Jakarta,https://www.mobil123.com/dijual/suzuki-s-press...
38,2022 Wuling EV Air ev Long Range Hatchback - 0...,-,Used,2022,Rp 188.000.000,Automatic,5 - 10K KM,Banten,https://www.mobil123.com/dijual/wuling-ev-air-...
63,2003 Toyota IST 1.5 Hatchback,-,Used,2003,Rp 70.000.000,Automatic,170 - 175K KM,Jawa Tengah,https://www.mobil123.com/dijual/toyota-ist-jaw...
92,2022 Daihatsu Rocky 1.2 X Wagon,-,Used,2022,Rp 160.000.000,Manual,25 - 30K KM,DKI Jakarta,https://www.mobil123.com/dijual/daihatsu-rocky...
...,...,...,...,...,...,...,...,...,...
26041,1979 Toyota Land Cruiser 4.2 Hardtop Jeep,-,Used,1979,Rp 148.000.000,Manual,150 - 155K KM,Jawa Timur,https://www.mobil123.com/dijual/toyota-land-cr...
26045,2005 Mercedes-Benz S280 2.8 Sedan,-,Used,2005,Rp 158.000.000,Automatic,75 - 80K KM,Jawa Timur,https://www.mobil123.com/dijual/mercedes-benz-...
26071,2007 Mitsubishi Lancer 1.8 1.8 GLXi Sedan - ce...,-,Used,2007,Rp 68.000.000,Manual,150000 KM,Jawa Barat,https://www.mobil123.com/dijual/mitsubishi-lan...
26110,2022 Wuling EV Air ev Charging Pile Long Range...,-,Used,2022,Rp 200.000.000,Automatic,19000 KM,Jawa Barat,https://www.mobil123.com/dijual/wuling-ev-air-...


There are lots of missing values in the "Car Name" column. Let's solve this problem by replacing all the original car name with the car name from the column "Title"

In [25]:
#there are car names with 2 and 3 words, so we'll separate the handling of each case.
#if the third word is not purely digit then we'll extract it, if not then we'll only extract the first 2 words.
two_words_pattern = r'\d{4}\s+([A-Za-z-]+)\s+([A-Za-z-]+)' #taking the first 2 words after year
three_words_pattern = r'\d{4}\s+([A-Za-z-]+)\s+([A-Za-z-]+)\s+([A-Za-z-]+)' #taking the first 3 words after year

#Extract car names
car_name_three_words = df2['Title'].str.extract(three_words_pattern, expand=False)
car_name_two_words = df2['Title'].str.extract(two_words_pattern, expand=False)

# Create columns for extracted data
df2['Car Name (Three Words)'] = car_name_three_words.apply(lambda row: ' '.join(row.dropna()), axis=1)
df2['Car Name (Two Words)'] = car_name_two_words.apply(lambda row: ' '.join(row.dropna()), axis=1)

# Check if the third word is purely digits
df2['Third Word'] = df2['Title'].str.split().str[3].fillna('')
df2['Is Digit with Period'] = df2['Third Word'].str.replace('.', '', 1).str.isdigit()

# Extract car names based on the condition
df2['Car Name'] = df2.apply(
    lambda row: ' '.join(row['Title'].split()[1:3]) if row['Is Digit with Period'] else ' '.join(row['Title'].split()[1:4]),
    axis=1
)

# Drop not used columns
df2 = df2.drop(columns=['Car Name (Three Words)', 'Car Name (Two Words)', 'Third Word', 'Is Digit with Period'])

df2["Car Name"].value_counts()

Car Name
Honda Brio                    2747
Toyota Calya                  1455
Suzuki S-Presso               1417
Toyota Agya                   1356
Toyota Avanza                 1350
Suzuki Ertiga                 1182
Daihatsu Sigra                 842
Suzuki Carry                   835
Daihatsu Ayla                  827
Wuling EV Air                  721
Honda Mobilio                  706
Daihatsu Xenia                 566
Toyota Yaris                   562
Mitsubishi Xpander             499
Toyota Kijang Innova           428
Honda CR-V                     372
Suzuki Ignis                   358
Daihatsu Terios                356
Toyota Sienta                  336
Nissan Grand Livina            334
Toyota Rush                    322
Honda Freed                    315
Wuling Confero                 306
Daihatsu Gran Max              298
Nissan X-Trail                 297
Suzuki Baleno                  293
Honda Jazz                     289
Honda BR-V                     273
Daihatsu Ro

In [26]:
(df2 == "-").sum()

Title               0
Car Name            0
Condition           0
Year                0
Price               0
Transmission        0
Total Kilometers    0
Location            0
Link                0
dtype: int64

### Car Brand

In [27]:
#make a new variable "Car Brand" by taking the first word of "Car Name" column
df2['Car Brand'] = df2['Car Name'].str.split().str[0]
df2['Car Brand'].value_counts()
#there might be some wrong brands in here because not all of the brands are placed in front of the car name, but it should be minor

Car Brand
Toyota           6756
Honda            5263
Suzuki           4855
Daihatsu         3469
Nissan           1454
Wuling           1353
Mitsubishi        865
Mazda             498
Chevrolet         310
Mercedes-Benz     188
Volkswagen        170
KIA               153
Ford              146
BMW               144
Hyundai           141
Isuzu              88
Datsun             84
Renault            81
DFSK               50
smart              42
Hino               21
Dodge              18
Jaguar             17
Subaru             16
MG                 16
Peugeot            15
Audi               15
Tata               11
Proton              6
Timor               5
Volvo               5
Opel                3
Lexus               3
Jeep                3
Willys              1
Chery               1
Geely               1
Citroen             1
CJ                  1
SsangYong           1
Aro                 1
Holden              1
Name: count, dtype: int64

### Change Price to integer

What to do:
* Remove "Rp"
* Remove periods
* Remove rows without any price
* Remove words after the price

In [28]:
df2['Price'].unique()

array(['Rp 109.000.000', 'Rp 195.000.000', 'Rp 115.000.000',
       'Rp 164.000.000', 'Rp 104.000.000', 'Rp 179.000.000',
       'Rp 191.000.000', 'Rp 145.000.000', 'Rp 143.400.000',
       'Rp 170.000.000', 'Rp 194.000.000', 'Rp 129.000.000',
       'Rp 148.000.000', 'Rp 143.000.000',
       'Rp 172.926.176\n\nHrg. Psrn.\n\n\n\n                                Mobil ini sedang Promo Hot Deals, karena harga rata-rata pasaran sekarang "Rp 172.926.176" dan sekarang diskon 5% di bawah harga pasaran, segera hubungi penjual sekarang!                            \n\n\n\n\n\n                -5% Rp 165.000.000',
       'Rp 149.000.000', 'Rp 150.000.003', 'Rp 163.100.000',
       'Rp 152.300.000', 'Rp 80.000.000', 'Rp 105.000.000',
       'Rp 119.000.000', 'Rp 122.500.000', 'Rp 171.000.000',
       'Rp 154.000.000', 'Rp 116.000.000', 'Rp 188.000.000',
       'Rp 140.000.000', 'Rp 189.000.000', 'Rp 110.000.000',
       'Rp 190.000.000', 'Rp 132.000.000', 'Rp 160.400.000',
       'Rp 142.989.898',


In [29]:
#define the pattern to extract the price
price_pattern = r'Rp\s+([\d,.]+)' #Rp followed by digits and period

#find all prices for each row
#for special case of hot deals, there are two prices, first is the non discounted price and last is the discounted price, we'll extract the last one.
df2['Price'] = df2['Price'].apply(lambda row: re.findall(price_pattern, row)[-1] if re.findall(price_pattern, row) else '0')

# Clean up the 'Price' column
df2['Price'] = df2['Price'].str.replace('.', '', regex=False)  # Remove periods
df2['Price'] = df2['Price'].str.extract('(\d+)', expand=False)  # Extract numeric part
df2['Price'] = pd.to_numeric(df2['Price'], errors='coerce')  # Convert to float
df2['Price'] = df2['Price'].fillna(0).astype(int)  # Fill NaNs with 0 and convert to integer

df2 = df2.loc[df2['Price'] != 335] # there's a price = 335, we'll assume it's 335M and drop it, because we only use price <= 200M Rupiah

df2['Price'].unique()
#price = 0 indicates a special offer which we have to contact the seller to know the actual price.

array([109000000, 195000000, 115000000, 164000000, 104000000, 179000000,
       191000000, 145000000, 143400000, 170000000, 194000000, 129000000,
       148000000, 143000000, 165000000, 149000000, 150000003, 163100000,
       152300000,  80000000, 105000000, 119000000, 122500000, 171000000,
       154000000, 116000000, 188000000, 140000000, 189000000, 110000000,
       190000000, 132000000, 160400000, 142989898,         0, 120000000,
       130000000,  75780000, 128000000, 175000000, 167000000,  70000000,
       192000000,  95000000, 183000000, 180000000, 139000000, 112000000,
        74000000, 157000000, 159000000, 160000000, 155000000, 185000000,
       135000000, 135000003, 147000000, 108000000, 200000000, 107000000,
       137000000, 163000000,   6000000, 197000000, 127000000,  20000000,
       169000000, 157500000,  12000000, 124000000, 118000000, 182000000,
         5000000,  99800000, 134640000, 134000000, 198900000, 199800000,
       131600000, 186750000, 162000000, 180400000, 

### Change Total Kilometers into category

What to do:
* Convert all the kilometers to integers (replace "K" with "000" and remove "KM").
* Create bins ranging from 0 to 200K KM in steps of 10K, with a final bin for values >200K KM.


Notes:

* If the value is a range, use the maximum total kilometers of the range.
* Cars for daily use typically have a range of 0 to 200K total kilometers, so we'll label all total kilometers greater than 200K as ">200K KM" because we won't really focus on these cars.

In [30]:
df2["Total Kilometers"].value_counts()
# - KM indicates a brand new car which we'll replace with 0 KM later on

Total Kilometers
0 - 5K KM          5537
- KM               1110
10 - 15K KM         745
60 - 65K KM         730
80 - 85K KM         714
50 - 55K KM         698
5 - 10K KM          675
40 - 45K KM         663
15 - 20K KM         662
20 - 25K KM         661
70 - 75K KM         656
75 - 80K KM         647
65 - 70K KM         644
85 - 90K KM         632
55 - 60K KM         622
25 - 30K KM         610
35 - 40K KM         607
30 - 35K KM         601
45 - 50K KM         587
90 - 95K KM         581
95 - 100K KM        545
100 - 105K KM       461
105 - 110K KM       364
115 - 120K KM       300
120 - 125K KM       290
110 - 115K KM       286
125 - 130K KM       202
130 - 135K KM       195
135 - 140K KM       162
140 - 145K KM       154
145 - 150K KM       114
295 - 300K KM       100
150 - 155K KM        94
90000 KM             86
160 - 165K KM        86
100000 KM            78
5000 KM              74
155 - 160K KM        72
80000 KM             72
120000 KM            69
1234 KM              64

In [31]:
#define the pattern
three_words_pattern = r'[A-Za-z0-9-]+\s+[-]+\s+([A-Za-z0-9-]+)' #

#remove KM
df2['Total Kilometers'] = df2['Total Kilometers'].str.replace('KM', '', regex=False).str.strip()

#variable to identify 'K' in a value
mask = df2['Total Kilometers'].str.contains('K', regex=False)

#Replace 'K' with '000' 
df2.loc[mask, 'Total Kilometers'] = df2.loc[mask, 'Total Kilometers'].str.replace('K', '000', regex=False)

#extract total km
df2['Total Kilometers'] = df2['Total Kilometers'].str.extract(three_words_pattern, expand=False).fillna(df2['Total Kilometers'])

#replace '-' with '0'
df2['Total Kilometers'] = df2['Total Kilometers'].str.replace('-', '0', regex=False) ## brand new cars is tagged with " - KM" so we'll change it to 0

#convert to int
df2['Total Kilometers'] = pd.to_numeric(df2['Total Kilometers'].str.replace(' ', ''), errors='coerce').astype(int)

df2['Total Kilometers'].value_counts()


Total Kilometers
5000       5613
0          1110
65000       760
15000       760
85000       751
55000       720
80000       719
90000       718
70000       702
45000       689
10000       687
20000       684
25000       682
75000       676
60000       664
40000       641
30000       638
50000       637
100000      623
35000       620
95000       616
105000      487
110000      422
120000      369
125000      312
115000      302
130000      245
135000      219
140000      208
145000      160
150000      138
300000      102
155000       96
165000       90
160000       84
1234         64
200000       60
185000       53
170000       53
205000       48
180000       45
175000       44
190000       38
195000       30
89000        28
58000        28
123456       26
27000        26
72000        25
64000        24
98000        24
84000        24
76000        23
94000        23
91000        23
78000        22
79000        22
108000       22
83000        22
81000        22
92000        22
117000 

In [32]:
#define the bins and labels for categorization
bins_km = list(range(0, 201000, 10000)) + [float('inf')]
labels_km = [f"{i}-{i+10}K KM" for i in range(0, 200, 10)] + [">200K KM"]

#categorize the 'Total Kilometers'
df2['Total Kilometers Category'] = pd.cut(df2['Total Kilometers'], bins=bins_km, labels=labels_km, right=True, include_lowest=True) #the range is left-exclusive and right-inclusive for example value of 60K will be in range of 50-60K KM
df2

,Title,Car Name,Condition,Year,Price,Transmission,Total Kilometers,Location,Link,Car Brand,Total Kilometers Category
0,2018 Toyota Calya 1.2 G MPV,Toyota Calya,Used,2018,109000000,Automatic,95000,Banten,https://www.mobil123.com/dijual/toyota-calya-g...,Toyota,90-100K KM
1,2024 Suzuki XL7 1.5 ALPHA Hybrid Wagon - PROMO...,Suzuki XL7,New,2024,195000000,Automatic,5000,Jawa Barat,https://www.mobil123.com/dijual/suzuki-xl7-alp...,Suzuki,0-10K KM
2,"2021 Suzuki XL7 1.5 BETA Wagon - 1,5",Suzuki XL7,Used,2021,195000000,Automatic,37538,Banten,https://www.mobil123.com/dijual/suzuki-xl7-bet...,Suzuki,30-40K KM
3,2011 Honda Jazz 1.5 Hatchback,Honda Jazz,Used,2011,115000000,Manual,135000,Banten,https://www.mobil123.com/dijual/honda-jazz-ban...,Honda,130-140K KM
4,2019 Toyota Avanza 1.3 G MPV,Toyota Avanza,Used,2019,164000000,Manual,60000,Jawa Barat,https://www.mobil123.com/dijual/toyota-avanza-...,Toyota,50-60K KM
...,...,...,...,...,...,...,...,...,...,...,...
26267,2015 Ford EcoSport 1.5 Titanium SUV - Tdp.15jt...,Ford EcoSport,Used,2015,130000000,Automatic,65000,DKI Jakarta,https://www.mobil123.com/dijual/ford-ecosport-...,Ford,60-70K KM
26268,2020 Toyota Avanza 1.3 E MPV - Manual,Toyota Avanza,Used,2020,175000000,Manual,35000,Sumatera Utara,https://www.mobil123.com/dijual/toyota-avanza-...,Toyota,30-40K KM
26269,"2018 Mitsubishi Xpander 1.5 ULTIMATE Wagon - 1,5",Mitsubishi Xpander,Used,2018,192000000,Automatic,56205,DKI Jakarta,https://www.mobil123.com/dijual/mitsubishi-xpa...,Mitsubishi,50-60K KM
26270,"2019 Daihatsu Sigra 1.2 R Deluxe MPV - 1,2",Daihatsu Sigra,Used,2019,125000000,Automatic,79645,Jawa Barat,https://www.mobil123.com/dijual/daihatsu-sigra...,Daihatsu,70-80K KM


### Change Price to category

In [33]:
#define the bins and labels for categorization
bins_price = list(range(0, 200001000, 50000000)) 
labels_price = [f"{i}-{i+50}M Rupiah" for i in range(0, 200, 50)]

#categorize the 'Price'
df2['Price Category'] = pd.cut(df2['Price'], bins=bins_price, labels=labels_price, right=True, include_lowest=True) #the range is left-exclusive and right-inclusive for example value of 50M will be in range of 0-50M Rupiah
df2

,Title,Car Name,Condition,Year,Price,Transmission,Total Kilometers,Location,Link,Car Brand,Total Kilometers Category,Price Category
0,2018 Toyota Calya 1.2 G MPV,Toyota Calya,Used,2018,109000000,Automatic,95000,Banten,https://www.mobil123.com/dijual/toyota-calya-g...,Toyota,90-100K KM,100-150M Rupiah
1,2024 Suzuki XL7 1.5 ALPHA Hybrid Wagon - PROMO...,Suzuki XL7,New,2024,195000000,Automatic,5000,Jawa Barat,https://www.mobil123.com/dijual/suzuki-xl7-alp...,Suzuki,0-10K KM,150-200M Rupiah
2,"2021 Suzuki XL7 1.5 BETA Wagon - 1,5",Suzuki XL7,Used,2021,195000000,Automatic,37538,Banten,https://www.mobil123.com/dijual/suzuki-xl7-bet...,Suzuki,30-40K KM,150-200M Rupiah
3,2011 Honda Jazz 1.5 Hatchback,Honda Jazz,Used,2011,115000000,Manual,135000,Banten,https://www.mobil123.com/dijual/honda-jazz-ban...,Honda,130-140K KM,100-150M Rupiah
4,2019 Toyota Avanza 1.3 G MPV,Toyota Avanza,Used,2019,164000000,Manual,60000,Jawa Barat,https://www.mobil123.com/dijual/toyota-avanza-...,Toyota,50-60K KM,150-200M Rupiah
...,...,...,...,...,...,...,...,...,...,...,...,...
26267,2015 Ford EcoSport 1.5 Titanium SUV - Tdp.15jt...,Ford EcoSport,Used,2015,130000000,Automatic,65000,DKI Jakarta,https://www.mobil123.com/dijual/ford-ecosport-...,Ford,60-70K KM,100-150M Rupiah
26268,2020 Toyota Avanza 1.3 E MPV - Manual,Toyota Avanza,Used,2020,175000000,Manual,35000,Sumatera Utara,https://www.mobil123.com/dijual/toyota-avanza-...,Toyota,30-40K KM,150-200M Rupiah
26269,"2018 Mitsubishi Xpander 1.5 ULTIMATE Wagon - 1,5",Mitsubishi Xpander,Used,2018,192000000,Automatic,56205,DKI Jakarta,https://www.mobil123.com/dijual/mitsubishi-xpa...,Mitsubishi,50-60K KM,150-200M Rupiah
26270,"2019 Daihatsu Sigra 1.2 R Deluxe MPV - 1,2",Daihatsu Sigra,Used,2019,125000000,Automatic,79645,Jawa Barat,https://www.mobil123.com/dijual/daihatsu-sigra...,Daihatsu,70-80K KM,100-150M Rupiah


### Save the cleaned dataset

In [494]:
df2.to_excel('mobil cleaned.xlsx',index=False)